## MLflow's Model Registry 
(Unnecessary to rerun it cause the versions were choosen manually looking at mlflow so if it is run again new versions will be created pointlesly)
To run this you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri=sqlite:///mlflow_db.db --default-artifact-root=s3://mlflow-artifacts-remote-jaime/`

In [2]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "http://127.0.0.1:5000"

# To instantiate it we need to pass a tracking URI and/or a registry URI
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.list_experiments()

[<Experiment: artifact_location='s3://mlflow-artifacts-remote-jaime/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>]

Let's check the latest versions for the experiment with id `2`...

In [3]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='4',
    # name='experiment-covid-1',
    # filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metric.evaluated_RMSE"]
)

In [4]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['evaluated_RMSE']:.4f}")

Comparing the experiments in mlflow the first one has less evaluated RSME so I will choose that model => RUNID = 16082a31f2be4eadb6f368b4ded2d309

### Interacting with the Model Registry

In this section I will use the `MlflowClient` instance to:

1. Register a new version for the experiment `experiment-covid-2`
2. Retrieve the latests versions of the model `covid-predictor` and check that a new version was created.


In [5]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [6]:
run_id = "ae2e389613094cf48d62eed43ce7850e" #This is not the run_id that will be chosen but it is to test that we can set it on stage "Archived"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="covid-predictor") #this is to register/create a new version of the model

Registered model 'covid-predictor' already exists. Creating a new version of this model...


RestException: RESOURCE_DOES_NOT_EXIST: Run with id=ae2e389613094cf48d62eed43ce7850e not found

In [ ]:
model_uri

'runs:/ae2e389613094cf48d62eed43ce7850e/model'

In [ ]:
model_name = "covid-predictor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 7, stage: None
version: 3, stage: Archived
version: 4, stage: Production


In [ ]:
model_version = 3
new_stage = "Archived"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1660136133272, current_stage='Archived', description='', last_updated_timestamp=1660143629972, name='covid-predictor', run_id='ae2e389613094cf48d62eed43ce7850e', run_link='', source='s3://mlflow-artifacts-remote-jaime/4/ae2e389613094cf48d62eed43ce7850e/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='3'>

#### Now we register the model that we want to use and set it in stage Production

In [ ]:
run_id = "16082a31f2be4eadb6f368b4ded2d309" #This is not the run_id that will be chosen but it is to test that we can set it on stage "Archived"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="covid-predictor") #this is to register/create a new version of the model

Registered model 'covid-predictor' already exists. Creating a new version of this model...
2022/08/10 15:00:30 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: covid-predictor, version 8
Created version '8' of model 'covid-predictor'.


<ModelVersion: creation_timestamp=1660143630182, current_stage='None', description='', last_updated_timestamp=1660143630182, name='covid-predictor', run_id='16082a31f2be4eadb6f368b4ded2d309', run_link='', source='s3://mlflow-artifacts-remote-jaime/4/16082a31f2be4eadb6f368b4ded2d309/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='8'>

In [ ]:
model_name = "covid-predictor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 8, stage: None
version: 3, stage: Archived
version: 4, stage: Production


In [ ]:
model_version = 4
new_stage = "Production"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1660136297566, current_stage='Production', description='Jrv_note: The model version 4 was transitioned to Production on 2022-08-10', last_updated_timestamp=1660143630858, name='covid-predictor', run_id='16082a31f2be4eadb6f368b4ded2d309', run_link='', source='s3://mlflow-artifacts-remote-jaime/4/16082a31f2be4eadb6f368b4ded2d309/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='4'>

In [ ]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"Jrv_note: The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: creation_timestamp=1660136297566, current_stage='Production', description='Jrv_note: The model version 4 was transitioned to Production on 2022-08-10', last_updated_timestamp=1660143631056, name='covid-predictor', run_id='16082a31f2be4eadb6f368b4ded2d309', run_link='', source='s3://mlflow-artifacts-remote-jaime/4/16082a31f2be4eadb6f368b4ded2d309/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='4'>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Download the model 
2. Test that the model works, make predictions and also check the rmse is low Load the test dataset, which corresponds to the last 7 days.
3. Download the model that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.

**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [ ]:
#Download model 
run_id = "16082a31f2be4eadb6f368b4ded2d309" 
client.download_artifacts(run_id=run_id, path='models', dst_path='.')

In [7]:
import pickle

with open("models/model.pkl", "rb") as f_in:
    model = pickle.load(f_in)

In [8]:
model

Pipeline(steps=[('poly', PolynomialFeatures(include_bias=False)),
                ('linear', LinearRegression())])

In [9]:
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from datetime import timedelta, datetime
from pandas._libs.tslibs.timestamps import Timestamp

TARGETS = ["ConfirmedCases", "Fatalities"]
features = ["prev_{}".format(col) for col in TARGETS]
loc_group = ["Province_State", "Country_Region"]

def preprocess(df):
    df["Date"] = df["Date"].astype("datetime64[ms]")
    for col in loc_group:
        df[col].fillna("none", inplace=True) #NOTE: replace all NaN with none  
    for col in TARGETS:
        df[col] = np.log1p(df[col]) 
    for col in TARGETS:
        df["prev_{}".format(col)] = df.groupby(loc_group)[col].shift() #NOTE: the prev_ columns basically has the same than the others but delayed one day
    return df

def get_data_last_days(num_days): #gets the data from the last "num_days" days
    num_days = num_days + 2 #I do this because I get rid of the first date since it has NaNs in the columns prev_ConfirmedCases	prev_Fatalities and because of the for loop with range
    dfs = []  # empty list which will hold your dataframes
    for d in range(1, num_days): #NOTE: do the same that has been done for the first day but for the whole period
        date = datetime.now() - timedelta(days=d)
        date_str = date.strftime("%m-%d-%Y")
        source_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/' + date_str + '.csv'
        df_temp = pd.read_csv(source_url)
        df_temp.rename(columns={"Last_Update": "Date"}, inplace=True) #Renane dataframe column from "Last_Update" to "Date"
        df_temp_2 = df_temp[["Admin2", "Province_State", "Country_Region","Confirmed", "Deaths"]].copy() #TODO: consider also other columns in future versions like Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
        df_temp_2.loc[:,"Date"] = date.strftime("%Y-%m-%d") 
        dfs.append(df_temp_2)  # append dataframe to list
    res = pd.concat(dfs, ignore_index=True)  # concatenate list of dataframes
    
    # group by Country_Region and sum Confirmed and Deaths
    df = res.groupby(['Province_State','Country_Region','Date']).agg({'Confirmed':'sum', 'Deaths':'sum'})
    df.reset_index(inplace=True)
    df.rename(columns={"Confirmed": "ConfirmedCases", "Deaths": "Fatalities"}, inplace=True)
        
    df = preprocess(df)
    
    df = df[df["Date"] > df["Date"].min()].copy() #removes the first day since it has NaNs in the "prev" columns

    df.reset_index(inplace=True, drop=True)
    
    return df

def predict_today_Province_State(model,Province_State):
    df = get_data_last_days(1) #Get data from yesterday
    y_pred = predict_today_world(model) #Predict today worldwide
    index_PS = df[df['Province_State']==Province_State].iloc[0].name
    predictions = y_pred[index_PS]
    return predictions #First the predicted Confirmed cases and second the predicted fatalities

def predict_today_world(model):#Does the prediction for today
    df = get_data_last_days(1) #Get data from yesterday
    yesterday = datetime.now() - timedelta(days=1)
    yesterday = yesterday.replace(hour=0, minute=0, second=0, microsecond=0)
    yesterday = Timestamp(yesterday)
    y_pred = np.clip(model.predict(df.loc[df["Date"] == yesterday][features]), None, 16)#NOTE: here predicting the targets for the first day and saturating (clip) them with max=16
    return y_pred


def evaluate_yesterday():
    return evaluate_last_days(1)

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def predict_past(model, num_days):
    test_df = get_data_last_days(num_days)
    first_day = datetime.now() - timedelta(days=num_days)
    first_day = first_day.replace(hour=0, minute=0, second=0, microsecond=0)
    first_day = Timestamp(first_day)
    y_pred = np.clip(model.predict(test_df.loc[test_df["Date"] == first_day][features]), None, 16)#NOTE: here he is predicting the targets for the first day and saturating (clip) them with max=16
 
    for i, col in enumerate(TARGETS):
        test_df["pred_{}".format(col)] = 0
        test_df.loc[test_df["Date"] == first_day, "pred_{}".format(col)] = y_pred[:, i] #NOTE: here he sets the predicted columns

    for d in range(1, num_days): #NOTE: do the same that has been done for the first day but for the whole period
        y_pred = np.clip(model.predict(y_pred), None, 16)
        date = first_day + timedelta(days=d)

    for i, col in enumerate(TARGETS):
        test_df.loc[test_df["Date"] == date, "pred_{}".format(col)] = y_pred[:, i]

    return test_df

def evaluate_last_days(model,num_days):
    #get data from the last "num_days" days
    df = predict_past(model,num_days)
    
    #get the rmse
    error = 0
    for col in TARGETS:
        error += rmse(df[col].values, df["pred_{}".format(col)].values) #NOTE: checks the error between the predicted columns and the target columns
    return np.round(error/len(TARGETS), 5)


In [14]:
evaluate_last_days(model,5)

/home/ubuntu/.local/share/virtualenvs/08-my_project-6zcrQ2k9/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(
/home/ubuntu/.local/share/virtualenvs/08-my_project-6zcrQ2k9/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(
/home/ubuntu/.local/share/virtualenvs/08-my_project-6zcrQ2k9/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(
/home/ubuntu/.local/share/virtualenvs/08-my_project-6zcrQ2k9/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(
/home/ubuntu/.local/share/virtualenvs/08-my_project-6zcrQ2k9/lib/python3.10/

7.54249

In [11]:
predict_today_Province_State(model,'Madrid')

array([14.45659418,  9.85487179])

In [12]:
ans = predict_today_Province_State(model,'Madrid')
df_ans = pd.DataFrame()
df_ans['pred_ConfirmedCases']=pd.Series(ans[0])
df_ans['pred_Fatalities']=pd.Series(ans[1])
df_ans

,pred_ConfirmedCases,pred_Fatalities
0,14.456594,9.854872


In [ ]:
model_name

Based on the results, the model that is currently on "Production" works fine (has low RMSE) so no need to change stage